In [3]:
import pandas as pd
import os

In [2]:
data_folder="../data"

In [49]:
aisles_df = pd.read_csv(os.path.join(data_folder, "aisles.csv"))
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [48]:
departments_df = pd.read_csv(os.path.join(data_folder, "departments.csv"))
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [54]:
product_df = pd.read_csv(os.path.join(data_folder, "products.csv"))
product_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [115]:
len(product_df)

49688

In [51]:
orders_meta_df = pd.read_csv(os.path.join(data_folder, "orders.csv"))
orders_meta_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [27]:
orders_train_df = pd.read_csv(os.path.join(data_folder, "order_products__train.csv"))
orders_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [44]:
def find_rating(row):
    #print(row)
    my_product_rating = 0
    if row[2] > 0:
        my_product_rating +=1
    if row[3] > 0:
        my_product_rating +=1
    return my_product_rating

In [50]:
orders_train_df['ratings'] = orders_train_df.apply(find_rating, axis=1)
orders_train_df.head()

,order_id,product_id,add_to_cart_order,reordered,ratings
0,1,49302,1,1,2
1,1,11109,2,1,2
2,1,10246,3,0,1
3,1,49683,4,0,1
4,1,43633,5,1,2


In [133]:
avg_dow_list = []
std_dow_list = []
avg_hour_list = []
std_hour_list = []

for idx in range(len(product_stats)):
    avg_dow_list.append(product_stats[idx][0])
    std_dow_list.append(product_stats[idx][1])
    avg_hour_list.append(product_stats[idx][2])
    std_hour_list.append(product_stats[idx][3])

for product in product_df.product_id:
    my_stats = []
    #print("Product ID: {}".format(product))
    
    # get list of reorders & orders
    is_my_product = orders_train_df['product_id'] == product
    is_reordered = orders_train_df['reordered'] > 0
    reorders_my_product = orders_train_df.loc[is_my_product & is_reordered].order_id.to_list()
    orders_my_product = orders_train_df.loc[is_my_product].order_id.to_list()
    
    # order df for order time info
    is_my_orders = orders_meta_df['order_id'].isin(orders_my_product)
    my_order_df = orders_meta_df.loc[is_my_orders]
    
    avg_dow = my_order_df.order_dow.mean()
    avg_dow_list.append(avg_dow)
    std_dow = my_order_df.order_dow.std()
    std_dow_list.append(std_dow)
     
    avg_hour = my_order_df.order_hour_of_day.mean()
    avg_hour_list.append(avg_hour)
    std_hour = my_order_df.order_hour_of_day.std()
    std_hour_list.append(std_hour)
    
print("{} products processed".format(len(avg_dow_list)))
print("test", avg_dow_list[:10])

In [ ]:
product_df["average_dow"] = avg_dow_list
product_df["standard_dev_dow"] = std_dow_list
product_df["average_hour_of_day"] = avg_hour_list
product_df["standard_dev_hour_of_day"] = std_hour_list

In [107]:
def get_days_between_orders(user_id, starting_order_number, ending_order_number):
    is_user = orders_meta_df.user_id == user_id
    is_not_too_low = orders_meta_df.order_number > starting_order_number                                        
    is_not_too_high = orders_meta_df.order_number <= ending_order_number

    return(orders_meta_df.loc[is_user & is_not_too_low & is_not_too_high].days_since_prior_order.sum())

# test should return 78.0
get_days_between_orders(1, 2, 5)

78.0

In [152]:
number_of_orders_list = []
for product in product_df.product_id:
    my_stats = []
    #print("Product ID: {}".format(product))
    
    is_my_product = orders_train_df['product_id'] == product
    orders_my_product = orders_train_df.loc[is_my_product].order_id.to_list()
    
    is_my_orders = orders_meta_df['order_id'].isin(orders_my_product)
    my_order_df = orders_meta_df.loc[is_my_orders]
    
    #print("\tNumber of orders: {}".format(len(my_order_df)))
    number_of_orders_list.append(len(my_order_df))
    
    # Checking for any re-order info
    # Spoiler alert: we had no re-orders in this data set
    '''
    days_since_reorder = None
    should_break=False
    for user in my_order_df.user_id:
        my_user_df = my_order_df.loc[my_order_df.user_id == user]

        if len(my_user_df) > 1:
            print("Someone re-ordered a product"!!!!!!", my_user_df)
            should_break=True
            break
                  
    if should_break:
        break
    '''
                  
print("{} products processed".format(len(number_of_orders_list)))
print("test", number_of_orders_list[:10])

49688 products processed
test [76, 4, 6, 22, 1, 0, 1, 13, 5, 119]


In [153]:
product_df["number_of_orders"] = number_of_orders_list

In [159]:
product_df.to_csv(os.path.join(data_folder, "products_added.csv"), index=False)

In [166]:
user_id_list = []

for idx, row in orders_train_df.iterrows():
    is_my_orders = orders_meta_df['order_id'] == row.order_id
    my_user = orders_meta_df.loc[is_my_orders].user_id.to_list()
    user_id_list.append(my_user[0])
    
print("{} products processed".format(len(user_id_list)))
print("Test", user_id_list[:10])

1384617 products processed
Test [112108, 112108, 112108, 112108, 112108, 112108, 112108, 112108, 79431, 79431]


In [167]:
orders_train_df['user_id'] = user_id_list
orders_train_df.head()

,order_id,product_id,add_to_cart_order,reordered,ratings,user_id
0,1,49302,1,1,2,112108
1,1,11109,2,1,2,112108
2,1,10246,3,0,1,112108
3,1,49683,4,0,1,112108
4,1,43633,5,1,2,112108


In [168]:
orders_train_df.to_csv(os.path.join(data_folder, "orders_added.csv"), index=False)